<a href="https://colab.research.google.com/github/azganushpoghosyan/mit_assignments/blob/main/ps5_news_triggers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def process(url):
    """
    Fetches news items from the rss url and parses them.
    Returns a list of NewsStory-s.
    """
    feed = feedparser.parse(url)
    entries = feed.entries
    ret = []
    for entry in entries:
        guid = entry.guid
        title = translate_html(entry.title)
        link = entry.link
        description = translate_html(entry.description)
        pubdate = translate_html(entry.published)

        try:
            pubdate = datetime.strptime(pubdate, "%a, %d %b %Y %H:%M:%S %Z")
            pubdate.replace(tzinfo=pytz.timezone("GMT"))
          #  pubdate = pubdate.astimezone(pytz.timezone('EST'))
          #  pubdate.replace(tzinfo=None)
        except ValueError:
            pubdate = datetime.strptime(pubdate, "%a, %d %b %Y %H:%M:%S %z")

        newsStory = NewsStory(guid, title, description, link, pubdate)
        ret.append(newsStory)
    return ret

###Problem 1

In [ ]:
class NewsStory(object):

  def __init__(guid, title, description, link, pubdate):
    self.guid = guid
    self.title = title
    self.description = description
    self.link = link
    self.pubdate = pubdate
  
  def get_guid(self):
    return self.guid

  def get_title(self):
    return self.title

  def get_description(self):
    return self.description

  def get_link(self):
    return self.link

  def get_pubdate(self):
    return self.pubdate

In [ ]:
class Trigger(object):

  def evaluate(self, story):

    """
    Returns True if an alert should be generated
    for the given news item, or False otherwise.
    """
    raise NotImplementedError

###Problem 2

In [ ]:
import string
class PhraseTrigger(Trigger):

  def __init__(self, phrase):
    self.phrase = phrase
  
  def is_phrase_in(self, text):
    phrase_clean = ' ' + self.phrase.lower() + ' '
    text_no_punc = ''.join(' ' if char in string.punctuation else char for char in text)
    text_clean = ' ' + ' '.join(text_no_punc.split()).lower() + ' '
    if phrase_clean in text_clean:
      return True
    else:
      return False

In [ ]:
obj1 = PhraseTrigger('purple cow')

In [ ]:
obj1.is_phrase_in('purplecowpurplecowpurplecow')

False

In [ ]:
class TitleTrigger(PhraseTrigger):
  def __init__(self, phrase):
    self.phrase = phrase
    PhraseTrigger.__init__(self, phrase)
  def evaluate(self, story):
    return self.is_phrase_in(self, story.get_title())

In [ ]:
class DescriptionTrigger(PhraseTrigger):
  def __init__(self, phrase):
    PhraseTrigger.__init__(self, phrase)
  def evaluate(self, story):
    return self.is_phrase_in(self, story.get_description())

In [ ]:
class TimeTrigger(Trigger):
  def __init__(self, time):
    self.time = datetime.strptime(time, "%d %b %Y %H:%M:%S").replace(tzinfo = pytz.timezone("EST"))

In [ ]:
class BeforeTrigger(TimeTrigger):
  def __init__(self, time):
    TimeTrigger.__init__(self, time)
  def evaluate(self, story):
    story_time = story.get()
    if story_time.tzinfo == None:
      story_time = story_time.replace(tzinfo = pytz.timezone("EST"))
    else:
      story_time = story_time.astimezone(tzinfo = pytz.timezone('EST'))
    return story_time < self.time

In [ ]:
class AfterTrigger(TimeTrigger):
  def __init__(self, time):
    TimeTrigger.__init__(self, time)
  def evaluate(self, story):
    story_time = story.get()
    if story_time.tzinfo == None:
      story_time = story_time.replace(tzinfo = pytz.timezone("EST"))
    else:
      story_time = story_time.astimezone(tzinfo = pytz.timezone('EST'))
    return story_time > self.time

In [ ]:
class NotTrigger(Trigger):
  def __init__(trigger):
    self.trigger = trigger
  def evaluate(self, story):
    return not self.trigger.evaluate(story)
  

In [ ]:
class AndTrigger(Trigger):
  def __init__(trigger1, trigger2):
    self.trigger1 = trigger1
    self.trigger2 = trigger2
  def evaluate(self, story):
    return self.trigger1.evaluate(story) and self.trigger2.evaluate(story)


In [ ]:
class OrTrigger(Trigger):
  def __init__(trigger1, trigger2):
    self.trigger1 = trigger1
    self.trigger2 = trigger2
  def evaluate(self, story):
    return self.trigger1.evaluate(story) or self.trigger2.evaluate(story)


In [ ]:
def filter_stories(stories, triggerlist):
  filtered_stories = []
  for trigger in triggerlist:
    for story in stories:
      if trigger.evaluate(story):
        filtered_stories.append(story)
  return filtered_stories

In [ ]:
def read_trigger_config(filename):
    """
    filename: the name of a trigger configuration file

    Returns: a list of trigger objects specified by the trigger configuration
        file.
    """
    # We give you the code to read in the file and eliminate blank lines and
    # comments. You don't need to know how it works for now!
    trigger_file = open(filename, 'r')
    lines = []
    for line in trigger_file:
        line = line.rstrip()
        if not (len(line) == 0 or line.startswith('//')):
            lines.append(line)

    # TODO: Problem 11
    # line is the list of lines that you need to parse and for which you need
    # to build triggers
    triggers_dict = {}
    for line in lines[0:-1]:
      line = line.split(',')
      if line[1] == 'TITLE':
        t = TitleTrigger(line[2])
      elif line[1] == 'DESCRIPTION':
        t = DescriptionTrigger(line[2])
      elif line[1] == 'AFTER':
        t = AfterTrigger(line[2])
      elif line[1] == 'BEFORE':
        t = AfterTrigger(line[2])
      elif line[1] == 'NOT':
        t = NotTrigger(line[2])
      elif line[1] == 'AND':
        t = AndTrigger(line[2], line[3])
      elif line[1] == 'OR':
        t = OrTrigger(line[2], line[3])
      triggers_dict[line[0]] = t
    triggers_list = []
    for trigger_key in lines[-1].split(',')[1::]:
      triggers_list.append(triggers_dict[trigger_key])
    #print(lines) # for now, print it so you see what it contains!
    return triggers_list

In [ ]:
SLEEPTIME = 120
def main_thread(master):
    # A sample trigger list - you might need to change the phrases to correspond
    # to what is currently in the news
    try:
        t1 = TitleTrigger("covid")
        t2 = DescriptionTrigger("Biden")
        t3 = DescriptionTrigger("China")
        t4 = AndTrigger(t2, t3)
        triggerlist = [t1, t4]

        # Problem 11
        # TODO: After implementing read_trigger_config, uncomment this line 
        triggerlist = read_trigger_config('triggers.txt')
        
        # HELPER CODE - you don't need to understand this!
        # Draws the popup window that displays the filtered stories
        # Retrieves and filters the stories from the RSS feeds
        frame = Frame(master)
        frame.pack(side=BOTTOM)
        scrollbar = Scrollbar(master)
        scrollbar.pack(side=RIGHT,fill=Y)

        t = "Google & Yahoo Top News"
        title = StringVar()
        title.set(t)
        ttl = Label(master, textvariable=title, font=("Helvetica", 18))
        ttl.pack(side=TOP)
        cont = Text(master, font=("Helvetica",14), yscrollcommand=scrollbar.set)
        cont.pack(side=BOTTOM)
        cont.tag_config("title", justify='center')
        button = Button(frame, text="Exit", command=root.destroy)
        button.pack(side=BOTTOM)
        guidShown = []
        def get_cont(newstory):
            if newstory.get_guid() not in guidShown:
                cont.insert(END, newstory.get_title()+"\n", "title")
                cont.insert(END, "\n---------------------------------------------------------------\n", "title")
                cont.insert(END, newstory.get_description())
                cont.insert(END, "\n*********************************************************************\n", "title")
                guidShown.append(newstory.get_guid())

        while True:

            print("Polling . . .", end=' ')
            # Get stories from Google's Top Stories RSS news feed
            stories = process("http://news.google.com/news?output=rss")

            # Get stories from Yahoo's Top Stories RSS news feed
            stories.extend(process("http://news.yahoo.com/rss/topstories"))

            stories = filter_stories(stories, triggerlist)

            list(map(get_cont, stories))
            scrollbar.config(command=cont.yview)


            print("Sleeping...")
            time.sleep(SLEEPTIME)

    except Exception as e:
        print(e)